<a href="https://colab.research.google.com/github/OnlyBart123/AI-Jwp/blob/main/AILab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
npz=np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int32)

npz=np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float32)
validation_targets = npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float32), npz['targets'].astype(np.int32)

In [ ]:
mean = np.mean(train_inputs, axis=0)
std = np.std(train_inputs, axis=0)

train_inputs = (train_inputs - mean) / std
validation_inputs = (validation_inputs - mean) / std
test_inputs = (test_inputs - mean) / std

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 250

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # zmiana
])

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size = 64
max_epochs=200
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)

class_weight = {
    0: 1.0,
    1: (train_targets == 0).sum() / (train_targets == 1).sum()  # np. jeśli jest 2x mniej jedynek, dostają wagę 2.0
}
print("Użyte class_weight:", class_weight)

test_accuracy = 0
run = 1

while test_accuracy < 0.9:
    print(f"\n🔁 Próba #{run} – trening modelu...")

    model.fit(
        train_inputs,
        train_targets,
        batch_size=batch_size,
        epochs=max_epochs,
        callbacks=[early_stopping],
        validation_data=(validation_inputs, validation_targets),
        class_weight=class_weight,
        verbose=1
    )

    _, test_accuracy = model.evaluate(test_inputs, test_targets, verbose=0)
    print(f"🎯 Test accuracy: {test_accuracy * 100:.2f}%")
    run += 1

Użyte class_weight: {0: 1.0, 1: np.float64(0.9894385769872152)}

🔁 Próba #1 – trening modelu...
Epoch 1/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6635 - loss: 0.5980 - val_accuracy: 0.7606 - val_loss: 0.4264
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7534 - loss: 0.4424 - val_accuracy: 0.7808 - val_loss: 0.4163
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7587 - loss: 0.4206 - val_accuracy: 0.7740 - val_loss: 0.4169
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7792 - loss: 0.3999 - val_accuracy: 0.7718 - val_loss: 0.3996
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7907 - loss: 0.3974 - val_accuracy: 0.7942 - val_loss: 0.3952
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8117 - loss: 0.3607 - val_accuracy: 0.8076 - val_loss: 0.3849
Epoch 7/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7920 - loss: 0.3846 - val_accuracy: 0.7919 - val_loss: 0.3914
Epoch 8/200
56

KeyboardInterrupt: 

In [111]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)
print('\n Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8290 - loss: 0.5547 

 Test loss: 0.66. Test accuracy: 80.58%
